## Day 25: The Halting Problem

Following the twisty passageways deeper and deeper into the CPU, you finally reach the core of the computer. Here, in the expansive central chamber, you find a grand apparatus that fills the entire room, suspended nanometers above your head.

You had always imagined CPUs to be noisy, chaotic places, bustling with activity. Instead, the room is quiet, motionless, and dark.

Suddenly, you and the CPU's **garbage collector** startle each other. "It's not often we get many visitors here!", he says. You inquire about the stopped machinery.

"It stopped milliseconds ago; not sure why. I'm a garbage collector, not a doctor." You ask what the machine is for.

### Part One

"Programs these days, don't know their origins. That's the **Turing machine**! It's what makes the whole computer work." You try to explain that Turing machines are merely models of computation, but he cuts you off. "No, see, that's just what they want you to think. Ultimately, inside every CPU, there's a Turing machine driving the whole thing! Too bad this one's broken. We're doomed!"

You ask how you can help. "Well, unfortunately, the only way to get the computer running again would be to create a whole new Turing machine from scratch, but there's no way you can-" He notices the look on your face, gives you a curious glance, shrugs, and goes back to sweeping the floor.

You find the **Turing machine blueprints** (your puzzle input) on a tablet in a nearby pile of debris. Looking back up at the broken Turing machine above, you can start to identify its parts:

 - A **tape** which contains 0 repeated infinitely to the left and right.
 
 - A **cursor**, which can move left or right along the tape and read or write values at its current position.
 
 - A set of **states**, each containing rules about what to do based on the current value under the cursor.

Each slot on the tape has two possible values: $0$ (the starting value for all slots) and $1$. Based on whether the cursor is pointing at a $0$ or a $1$, the current state says what value to write at the current position of the cursor, whether to move the cursor left or right one slot, and which state to use next.

For example, suppose you found the following blueprint:

    Begin in state A.
    Perform a diagnostic checksum after 6 steps.

    In state A:
      If the current value is 0:
        - Write the value 1.
        - Move one slot to the right.
        - Continue with state B.
      If the current value is 1:
        - Write the value 0.
        - Move one slot to the left.
        - Continue with state B.

    In state B:
      If the current value is 0:
        - Write the value 1.
        - Move one slot to the left.
        - Continue with state A.
      If the current value is 1:
        - Write the value 1.
        - Move one slot to the right.
        - Continue with state A.

Running it until the number of steps required to take the listed diagnostic checksum would result in the following tape configurations (with the cursor marked in square brackets):

... 0  0  0 [0] 0  0 ... (before any steps; about to run state A)
... 0  0  0  1 [0] 0 ... (after 1 step;     about to run state B)
... 0  0  0 [1] 1  0 ... (after 2 steps;    about to run state A)
... 0  0 [0] 0  1  0 ... (after 3 steps;    about to run state B)
... 0 [0] 1  0  1  0 ... (after 4 steps;    about to run state A)
... 0  1 [1] 0  1  0 ... (after 5 steps;    about to run state B)
... 0  1  1 [0] 1  0 ... (after 6 steps;    about to run state A)

The CPU can confirm that the Turing machine is working by taking a diagnostic checksum after a specific number of steps (given in the blueprint). Once the specified number of steps have been executed, the Turing machine should pause; once it does, count the number of times 1 appears on the tape. In the above example, the diagnostic checksum is 3.

Recreate the Turing machine and save the computer! What is the diagnostic checksum it produces once it's working again?

In [34]:
from collections import defaultdict

class Tape(defaultdict):
    
    def __init__(self):
        super().__init__(int)
        self.lower_bound = 0
        self.upper_bound = 0
        
        
    def __getitem__(self, k):
        assert type(k) is int
        self.lower_bound = min(k, self.lower_bound)
        self.upper_bound = max(k, self.upper_bound)
        return super().__getitem__(k)
    
    def __setitem__(self, k, v):
        assert type(k) is int
        assert v in (0, 1)
        self.upper_bound = max(k, self.upper_bound)
        self.lower_bound = min(k, self.lower_bound)
        return super().__setitem__(k, v)
    
    def dump(self, cursor=0):
        length = self.upper_bound - self.lower_bound +1
        for _ in range(length):
            print('---', end='+')
        print()
        for pos in range(self.lower_bound, self.upper_bound+1):
            if pos in (-1, 0):
                separator = '║'
            else:
                separator = '|'
            v = self[pos]
            if pos == cursor:
                print('[{}]'.format(v), end=separator)
            else:
                print('{:^3}'.format(v), end=separator)
        print()
        for _ in range(length):
            print('---', end='+')
        print()    

        
tape = Tape()
tape[-3] == 0
tape[4] = 1
tape.lower_bound == -3
tape.upper_bound == 4
tape.dump()
print('ok')

---+---+---+---+---+---+---+---+
 0 | 0 | 0 ║[0]║ 0 | 0 | 0 | 1 |
---+---+---+---+---+---+---+---+
ok


In [35]:
class TuringMachine:
    
    def __init__(self):
        self.cursor = 0
        self.tape = Tape()
        
    def left(self):
        self.cursor -= 1

    def right(self):
        self.cursor += 1
        
    def write(self, v):
        self.tape[self.cursor] = v
        
    def read(self):
        return self.tape[self.cursor]
    
    def dump(self):
        self.tape.dump(self.cursor)

    

tm = TuringMachine()
assert tm.cursor == 0
tm.left()
tm.write(1)
assert tm.read() == 1
assert tm.cursor == -1
tm.right()
assert tm.cursor == 0
tm.dump()
print('ok')

---+---+
 1 ║[0]║
---+---+
ok


In [36]:
import re

pat_rule = re.compile('R[0|1]W[0|1]M[L|R]G[A-F]')

# R R(ead) 0|1 W(rite) 0|1 M(ove) L(eft)|R(ight) G(o state) [A-F]

class State:
    
    def __init__(self, name, r0, r1):
        self.name = name
        assert r0.startswith('R0')
        assert pat_rule.match(r0)
        assert r1.startswith('R1')
        assert pat_rule.match(r1)
        self.rules = [r0, r1]
        
    def step(self, tm):
        cur_val = tm.read()
        rule = self.rules[cur_val]
        tm.write(int(rule[3]))
        if rule[5] == 'L':
            tm.left()
        elif rule[5] == 'R':
            tm.right()
        return rule[7]
    

States = {
    'A': State('A', 'R0W1MRGB', 'R1W0MLGC'),
    'B': State('B', 'R0W1MLGA', 'R1W1MRGD'),
    'C': State('C', 'R0W0MLGB', 'R1W0MLGE'),
    'D': State('D', 'R0W1MRGB', 'R1W0MRGB'),
    'E': State('E', 'R0W1MLGF', 'R1W1MLGC'),
    'F': State('F', 'R0W1MRGD', 'R1W1MRGA')
}

In [38]:
# tests

States = {
    'A': State('A', 'R0W1MRGB', 'R1W0MLGB'),
    'B': State('B', 'R0W1MLGA', 'R1W1MRGA'),
}
 
state = States['A']
tm = TuringMachine()
tm.dump()
for i in range(6):
    new_state_label = state.step(tm)
    state = States[new_state_label]
    tm.dump()
    

---+
[0]║
---+
---+
 1 ║
---+
---+---+
[1]║ 1 |
---+---+
---+---+
 0 ║ 1 |
---+---+
---+---+---+
 1 ║ 0 ║ 1 |
---+---+---+
---+---+---+---+
 1 |[1]║ 0 ║ 1 |
---+---+---+---+
---+---+---+---+
 1 | 1 ║[0]║ 1 |
---+---+---+---+


In [43]:
# The real thing

States = {
    'A': State('A', 'R0W1MRGB', 'R1W0MLGC'),
    'B': State('B', 'R0W1MLGA', 'R1W1MRGD'),
    'C': State('C', 'R0W0MLGB', 'R1W0MLGE'),
    'D': State('D', 'R0W1MRGA', 'R1W0MRGB'),
    'E': State('E', 'R0W1MLGF', 'R1W1MLGC'),
    'F': State('F', 'R0W1MRGD', 'R1W1MRGA')
}
state = States['A']
tm = TuringMachine()
for i in range(12481997):
    new_state_label = state.step(tm)
    state = States[new_state_label]

In [42]:
print('Part one:', sum(tm.tape.values()))

Part one: 3362


### Part Two

The Turing machine, and soon the entire computer, springs back to life. A console glows dimly nearby, awaiting your command.

> reboot printer
Error: That command requires priority 50. You currently have priority 0.
You must deposit 50 stars to increase your priority to the required level.

The console flickers for a moment, and then prints another message:

Star accepted.
You must deposit 49 stars to increase your priority to the required level.

The garbage collector winks at you, then continues sweeping.

You have enough stars to
.